## Synthetic Experiments

In this notebook, we generate synthetic data sets with a ground truth item response theory model and test how often the bounds returned by various confidence bound methods actually include the 'true' ability parameter and how precisely the bound estimates the difference between true parameter and estimated parameter.

In [1]:
# set up data generation function
def sample_data(m, n):
    theta = np.random.randn(m)
    b     = np.random.randn(n)
    P     = 1. / (1. + np.exp(-(np.expand_dims(theta, 1) - np.expand_dims(b, 0))))
    X     = np.random.rand(m, n)
    X[X >= 1. - P] = 1.
    X[X <  1. - P] = 0.
    return theta, b, P, X
# set up a function to evaluate coverage
def eval_coverage(theta, theta_min, theta_max):
    return np.mean(np.logical_and(theta >= theta_min, theta <= theta_max))
# set up a function to compare the bound size with the size needed to cover the true theta
def eval_logbias(theta, theta_est, theta_min, theta_max):
    ratios  = np.zeros_like(theta)
    small = theta < theta_est
    lo    = theta_est[small] - theta_min[small]
    ratios[small] = lo / (theta_est[small] - theta[small])
    large = theta >= theta_est
    hi    = theta_max[large] - theta_est[large]
    ratios[large] = hi / (theta[large] - theta_est[large])
    return np.mean(np.log(ratios))

In [2]:
# set up experimental hyper-parameters
experimental_conditions = [
    (30, 10),
    (30, 20),
    (50, 10),
    (50, 20),
    (100, 10),
    (100, 20),
    (500, 10),
    (500, 20)
]
R     = 10

regul = 1.
alpha = .95
from scipy.stats import chi2
absolute_bound = .5 * chi2.ppf(alpha, df = 1)
mu    = .01

method_labels = ['wald', 'likelihood-profile', 'barrier', 'AO(1)', 'AO(2)', 'AO(3)']

In [3]:
# perform experiment in varying conditions
import numpy as np
from tqdm import tqdm
import time
import ability_bounds

for i in range(len(experimental_conditions)):
    m, n = experimental_conditions[i]
    print('--- condition %d; m = %d, n = %d ---' % (i+1, m, n))

    coverage = np.zeros((len(method_labels), R))
    logbias  = np.zeros((len(method_labels), R))
    runtimes = np.zeros((len(method_labels), R))

    for r in tqdm(range(R)):
        # sample new data set
        theta, b, P, X = sample_data(m, n)
        # iterate over all methods
        for method in range(len(method_labels)):
            # set up a fresh model
            if method_labels[method] == 'wald':
                model = ability_bounds.WaldBounds(regul, alpha)
            elif method_labels[method] == 'likelihood-profile':
                model = ability_bounds.LikelihoodProfile(regul, alpha)
            elif method_labels[method] == 'barrier':
                model = ability_bounds.BarrierBounds(regul, absolute_bound = absolute_bound)
            elif method_labels[method].startswith('AO'):
                num_iterations = int(method_labels[method][3])
                model = ability_bounds.AOBounds(regul, absolute_bound = absolute_bound, num_iterations = num_iterations)
            else:
                raise ValueError('unknown method: %s' % method_labels[method])
            # fit the model to the data
            start = time.time()
            model.fit(X)
            runtimes[method, r] = time.time() - start
            # evaluate the model
            coverage[method, r] = eval_coverage(theta, model.theta_min_, model.theta_max_)
            logbias[method, r]  = eval_logbias(theta, model.theta_, model.theta_min_, model.theta_max_)

    # print current results
    print('method              \tcoverage\tlogbias\t\truntime')
    for method in range(len(method_labels)):
        row = method_labels[method] + (20 - len(method_labels[method])) * ' '
        for measure in [coverage, logbias, runtimes]:
            row += '\t%.3f +- %.3f' % (np.mean(measure[method, :]), np.std(measure[method, :]))
        print(row)
    # store current results
    filename = 'results_%d_%d.csv' % (m, n)
    datamat  = np.concatenate((coverage.T, logbias.T, runtimes.T), 1)
    header   = []
    for measure in ['coverage', 'logbias', 'runtime']:
        for method_label in method_labels:
            header.append('%s_%s' % (measure, method_label))
    np.savetxt(filename, datamat, delimiter = '\t', fmt = '%g', header = '\t'.join(header), comments = '')

--- condition 1; m = 30, n = 10 ---


100%|██████████| 10/10 [00:06<00:00,  1.50it/s]


method              	coverage	logbias		runtime
wald                	0.983 +- 0.022	1.721 +- 0.177	0.005 +- 0.000
likelihood-profile  	0.943 +- 0.054	1.280 +- 0.179	0.316 +- 0.008
barrier             	0.910 +- 0.056	1.113 +- 0.179	0.045 +- 0.001
AO(1)               	0.943 +- 0.054	1.260 +- 0.179	0.020 +- 0.000
AO(2)               	0.943 +- 0.054	1.280 +- 0.179	0.108 +- 0.002
AO(3)               	0.943 +- 0.054	1.280 +- 0.179	0.171 +- 0.002
--- condition 2; m = 30, n = 20 ---


  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-1-178cdb071133>:22: RuntimeWarning: divide by zero encountered in log
  return np.mean(np.log(ratios))
100%|██████████| 10/10 [00:07<00:00,  1.41it/s]
/usr/lib64/python3.10/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


method              	coverage	logbias		runtime
wald                	1.000 +- 0.000	1.988 +- 0.186	0.006 +- 0.001
likelihood-profile  	0.943 +- 0.021	1.339 +- 0.201	0.333 +- 0.005
barrier             	0.083 +- 0.050	-inf +- nan	0.035 +- 0.001
AO(1)               	0.940 +- 0.025	1.305 +- 0.202	0.021 +- 0.001
AO(2)               	0.943 +- 0.021	1.339 +- 0.201	0.118 +- 0.002
AO(3)               	0.943 +- 0.021	1.339 +- 0.201	0.193 +- 0.002
--- condition 3; m = 50, n = 10 ---


...

100%|██████████| 10/10 [05:01<00:00, 30.16s/it]


method              	coverage	logbias		runtime
wald                	0.998 +- 0.002	1.837 +- 0.068	0.145 +- 0.008
likelihood-profile  	0.940 +- 0.011	1.318 +- 0.060	16.868 +- 1.047
barrier             	0.898 +- 0.015	1.170 +- 0.068	1.924 +- 0.066
AO(1)               	0.939 +- 0.010	1.316 +- 0.060	0.410 +- 0.012
AO(2)               	0.940 +- 0.011	1.318 +- 0.060	4.605 +- 0.266
AO(3)               	0.940 +- 0.011	1.318 +- 0.060	6.205 +- 0.284
--- condition 8; m = 500, n = 20 ---


 40%|████      | 4/10 [03:20<05:02, 50.38s/it]<ipython-input-1-178cdb071133>:22: RuntimeWarning: divide by zero encountered in log
  return np.mean(np.log(ratios))
 50%|█████     | 5/10 [04:10<04:10, 50.16s/it]<ipython-input-1-178cdb071133>:22: RuntimeWarning: divide by zero encountered in log
  return np.mean(np.log(ratios))
 80%|████████  | 8/10 [06:42<01:40, 50.31s/it]<ipython-input-1-178cdb071133>:22: RuntimeWarning: divide by zero encountered in log
  return np.mean(np.log(ratios))
 90%|█████████ | 9/10 [07:33<00:50, 50.58s/it]<ipython-input-1-178cdb071133>:22: RuntimeWarning: divide by zero encountered in log
  return np.mean(np.log(ratios))
100%|██████████| 10/10 [08:23<00:00, 50.35s/it]

method              	coverage	logbias		runtime
wald                	1.000 +- 0.001	2.065 +- 0.044	0.230 +- 0.017
likelihood-profile  	0.949 +- 0.008	1.314 +- 0.044	29.796 +- 0.735
barrier             	0.097 +- 0.016	-inf +- nan	1.857 +- 0.036
AO(1)               	0.948 +- 0.008	1.310 +- 0.044	0.479 +- 0.023
AO(2)               	0.949 +- 0.008	1.314 +- 0.044	8.010 +- 0.189
AO(3)               	0.949 +- 0.008	1.314 +- 0.044	9.983 +- 0.214



/usr/lib64/python3.10/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
